# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
import pandas as pd
import numpy as np
from xml.etree import ElementTree as ET

/Users/tanzinazaman/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
#Problem 1: 10 countries with the lowest infant mortality rates
def iter_name(etree):
    name_list = [];
    for country in etree.iterfind('country'):
        name = country.find('name').text
        mortality = "N/A"
        if country.find('infant_mortality') is not None:
            mortality = float(country.find('infant_mortality').text)
        pair = [name, mortality]
        name_list.append(pair)
    #print name_list
    return name_list
        
        
etree = ET.parse( './data/mondial_database.xml' ) #create an ElementTree object 
name_list = iter_name(etree)

mortality_rate_df = pd.DataFrame(name_list, columns = ['country_name', 'mortality_rate'])
answer1 = mortality_rate_df.sort_values('mortality_rate',ascending = True)
answer1.head(10)

,country_name,mortality_rate
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.6
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [3]:
# Problem 2: 10 cities with most population
city=[]
population=[]

for element in etree.iterfind('country'):
    for subelement in element.getiterator('city'):
        city.append(subelement.find('name').text)
        populate=''
        for node in subelement.iter('population'):
            populate=node.text
        population.append(populate)
               
data = pd.DataFrame()
data['city']=city
data['population']=population
data['population']=data['population'].replace('',np.nan)
data['population']=data['population'].astype(float)
data.sort_values(by='population', ascending=False).head(10)

,city,population
1341,Shanghai,22315474.0
771,Istanbul,13710512.0
1527,Mumbai,12442373.0
479,Moskva,11979529.0
1340,Beijing,11716620.0
2810,São Paulo,11152344.0
1342,Tianjin,11090314.0
1064,Guangzhou,11071424.0
1582,Delhi,11034555.0
1067,Shenzhen,10358381.0


In [4]:
# Problem 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
df = pd.DataFrame(columns=['country','ethnicgroup','population','percentage'])
for element in etree.iterfind('country'):
    for subelement in element.getiterator('ethnicgroup'):
        country=element.find('name').text
        ethnicgroup=subelement.text
        population=element.find('population').text
        percentage=subelement.attrib['percentage']
        
        df2=pd.DataFrame([[country,ethnicgroup,population,percentage]], 
                         columns=['country','ethnicgroup','population','percentage'])
        df = df.append(df2)

df['population']=pd.to_numeric(df['population'],errors='coerce')   
df['percentage']=pd.to_numeric(df['percentage'],errors='coerce') 
df['ethnic_population']=df['population']*df['percentage']/100
df.groupby(['ethnicgroup']).sum().sort_values(by='ethnic_population',ascending=False).head(10)

,population,percentage,ethnic_population
ethnicgroup,,,
Han Chinese,543776080,91.50,4.975551e+08
European,362717873,970.82,1.928658e+08
Indo-Aryan,238396327,72.00,1.716454e+08
Russian,202263854,224.10,9.275844e+07
African,357529690,1868.55,8.632937e+07
Japanese,82199470,99.40,8.170627e+07
German,145710759,165.60,6.623219e+07
Dravidian,238396327,25.00,5.959908e+07
English,50616012,83.60,4.231499e+07
